# Data Preprocessing for Treatment Group 

Treatment Group: 20 sites (activated on December 1, 2022)

In [1]:
# Setup
import os
import pandas as pd
import numpy as np
from haversine import haversine
from datetime import datetime

os.chdir("C:/Users/h2408/Downloads/Applied Econometrics/term paper") #change working directory
startMonth = '2022-06'
endMonth = '2023-12'
filter_accidentType = 'all' #'all' , 'A1', 'A2'
distance_threshold = 0.01
fileName = 'workdata/Treatment_10m_all.csv'

In [2]:
equipment = pd.read_csv('rawdata/臺北市智慧管理科技執法設備資料表.csv')
used_type = '路口多功能'
used_time = '111年12月1日'
year2months = pd.period_range(start=startMonth, end=endMonth, freq='M')
# filter data
used_equipment = equipment.loc[
    (equipment['名稱'] == used_type) &
    (equipment['啟用日期'] == used_time) 
]
# rename columns
used_equipment.columns = ['id', 'type', 'address', 'district', 'longitude', 'latitude', 'status', 'item']
used_equipment.reset_index(drop=True, inplace=True)
used_equipment.head()

,id,type,address,district,longitude,latitude,status,item
0,6,路口多功能,市民大道與承德路口,中正區、大同區,121.516415,25.048688,111年12月1日,闖紅燈、不停讓行人、不依規定轉彎、不依標誌標線號誌指示行駛、違規停車
1,7,路口多功能,民權西路與承德路口,大同區,121.518198,25.063106,111年12月1日,闖紅燈、不依規定轉彎、不依標誌標線號誌指示行駛
2,8,路口多功能,民權西路與延平北路口,大同區,121.511237,25.062788,111年12月1日,闖紅燈、不停讓行人、不依規定轉彎、不依標誌標線號誌指示行駛、汽車行駛機車道(上匝)
3,9,路口多功能,民權西路與蘭州街口,大同區,121.514742,25.062957,111年12月1日,闖紅燈、不依規定轉彎
4,10,路口多功能,大直街與北安路口,中山區,121.549383,25.084289,111年12月1日,闖紅燈、不依規定轉彎、不依標誌標線號誌指示行駛、違規停車


In [3]:
# accident110 = pd.read_csv('rawdata/110年臺北市道路交通事故斑點圖(改A1A2).csv', encoding='big5')
accident111 = pd.read_csv('rawdata/111年臺北市道路交通事故斑點圖(改A1A2).csv', encoding='big5')
accident112 = pd.read_csv('rawdata/112年臺北市道路交通事故斑點圖.csv', encoding='big5')
accidentCombined = pd.concat([accident111, accident112])
# rename columns
accidentCombined.columns = ['accident_time', 'accident_type', 'accident_location', 'accident_longitude', 'accident_latitude']
# filter data
if filter_accidentType != 'all':
    accidentCombined = accidentCombined.loc[
        (accidentCombined['accident_type'] == filter_accidentType) 
    ]
accidentCombined['accident_time'] = pd.to_datetime(accidentCombined['accident_time'], errors='coerce')
accidentCombined['accident_month'] = accidentCombined['accident_time'].dt.to_period('M')
accidentCombined.head()

,accident_time,accident_type,accident_location,accident_longitude,accident_latitude,accident_month
0,2022-01-01 00:37:00,2,松山區復興北路233號,121.544239,25.057154,2022-01
1,2022-01-01 00:42:00,2,大安區忠孝東路3段與建國南路1段口,121.537449,25.041553,2022-01
2,2022-01-01 02:13:00,2,萬華區西寧南路與貴陽街2段口,121.506210,25.039430,2022-01
3,2022-01-01 02:24:00,2,中山區民權東路1段與雙城街口,121.524356,25.062853,2022-01
4,2022-01-01 02:58:00,2,中正區中山南路與貴陽街1段口,121.517320,25.038707,2022-01


In [4]:
# accident detail
accidentDetail111 = pd.read_csv('rawdata/111年-臺北市A1及A2類交通事故明細.csv', encoding='big5')
accidentDetail112 = pd.read_csv('rawdata/112年-臺北市A1及A2類交通事故明細.csv', encoding='big5')
accidentDetailCombined = pd.concat([accidentDetail111, accidentDetail112])
# used columns
# 生成 time 欄位（轉換 ROC 年份為公元年）
def generate_time(row):
    year = row['發生年度'] + 1911
    return datetime(year, row['發生月'], row['發生日'], row['發生時-Hours'], row['發生分'], 0)
accidentDetailCombined['accident_time'] = accidentDetailCombined.apply(generate_time, axis=1)
accidentDetailCombined = accidentDetailCombined[['accident_time', '肇事地點', '死亡人數', '2-30日死亡人數', '受傷人數', '4天候', '5光線', '6道路類別', '7速限-速度限制', '8道路型態', '9事故位置', '15事故類型及型態', '肇因碼-主要']]
# rename columns
accidentDetailCombined.columns = ['accident_time', 'accident_location', 'death', 'death_2_30', 'injury', 'weather', 'light', 'road_type', 'speed_limit', 'road_shape', 'accident_place', 'accident_type', 'cause']
# filter data
accidentDetailCombined = accidentDetailCombined.groupby(['accident_time', 'accident_location']).first().reset_index()
# merge data
merged_data = pd.merge(accidentCombined, accidentDetailCombined, on=['accident_time', 'accident_location'], how='left')
merged_data.head()

C:\Users\h2408\AppData\Local\Temp\ipykernel_32668\1287404237.py:2: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  accidentDetail111 = pd.read_csv('rawdata/111年-臺北市A1及A2類交通事故明細.csv', encoding='big5')
C:\Users\h2408\AppData\Local\Temp\ipykernel_32668\1287404237.py:3: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  accidentDetail112 = pd.read_csv('rawdata/112年-臺北市A1及A2類交通事故明細.csv', encoding='big5')


,accident_time,accident_type_x,accident_location,accident_longitude,accident_latitude,accident_month,death,death_2_30,injury,weather,light,road_type,speed_limit,road_shape,accident_place,accident_type_y,cause
0,2022-01-01 00:37:00,2,松山區復興北路233號,121.544239,25.057154,2022-01,0.0,0.0,2.0,7.0,3.0,5.0,50.0,14.0,9.0,17.0,23.0
1,2022-01-01 00:42:00,2,大安區忠孝東路3段與建國南路1段口,121.537449,25.041553,2022-01,0.0,0.0,3.0,7.0,3.0,5.0,50.0,4.0,1.0,15.0,7.0
2,2022-01-01 02:13:00,2,萬華區西寧南路與貴陽街2段口,121.506210,25.039430,2022-01,0.0,0.0,1.0,8.0,3.0,5.0,50.0,4.0,1.0,16.0,44.0
3,2022-01-01 02:24:00,2,中山區民權東路1段與雙城街口,121.524356,25.062853,2022-01,0.0,0.0,1.0,8.0,3.0,5.0,50.0,3.0,1.0,17.0,44.0
4,2022-01-01 02:58:00,2,中正區中山南路與貴陽街1段口,121.517320,25.038707,2022-01,0.0,0.0,1.0,8.0,3.0,5.0,50.0,4.0,2.0,18.0,43.0


In [5]:
# merge accident data with equipment data
merged_data.describe()

,accident_time,accident_type_x,accident_longitude,accident_latitude,death,death_2_30,injury,weather,light,road_type,speed_limit,road_shape,accident_place,accident_type_y,cause
count,49696,49696.000000,49695.000000,49695.000000,49683.000000,49683.000000,49683.000000,25424.000000,25424.000000,25424.000000,25424.000000,25424.000000,25424.000000,25424.000000,24052.000000
mean,2022-12-28 22:28:47.213055488,1.997525,121.540593,25.054781,0.002476,0.000704,1.332407,7.350928,1.622679,5.038546,43.753383,7.698238,4.518880,14.085706,18.535714
min,2022-01-01 00:37:00,1.000000,121.453876,24.964706,0.000000,0.000000,0.000000,1.000000,1.000000,3.000000,0.000000,3.000000,1.000000,1.000000,1.000000
25%,2022-07-09 10:26:45,2.000000,121.517225,25.034579,0.000000,0.000000,1.000000,7.000000,1.000000,5.000000,40.000000,4.000000,1.000000,12.000000,7.000000
50%,2022-12-25 13:58:30,2.000000,121.536983,25.050293,0.000000,0.000000,1.000000,8.000000,1.000000,5.000000,50.000000,4.000000,2.000000,16.000000,15.000000
75%,2023-06-19 20:53:15,2.000000,121.562291,25.070747,0.000000,0.000000,2.000000,8.000000,3.000000,5.000000,50.000000,14.000000,9.000000,17.000000,23.000000
max,2023-12-31 23:23:00,2.000000,121.661683,25.188117,1.000000,1.000000,8.000000,8.000000,4.000000,8.000000,80.000000,17.000000,22.000000,29.000000,67.000000
std,NaN,0.049689,0.030677,0.033542,0.049695,0.026533,0.598350,0.854544,0.921225,0.324450,11.140885,4.970561,4.280139,3.915734,13.514033


In [6]:
# 計算每個設備在每月distance_threshold公尺範圍內的交通事故數量, injury數量；紀錄'weather'為list的資料
accident_count = []  # 儲存每個設備的交通事故數量
injury_count = []  # 儲存每個設備的傷亡人數
weather_records = []  # 儲存每個設備的天氣資訊（列表）
for i, equipment in used_equipment.iterrows():
    monthly_accident_counts = []
    monthly_injury_counts = []
    monthly_weather_records = []
    for year2month in year2months:
        # 篩選當前月份的交通事故
        monthly_accidents = merged_data[merged_data['accident_month'] == year2month].copy()
        
        # 計算距離並篩選事故
        monthly_accidents['distance'] = monthly_accidents.apply(
            lambda x: haversine((equipment['latitude'], equipment['longitude']), 
                                (x['accident_latitude'], x['accident_longitude'])),
            axis=1
        )
        nearby_accidents = monthly_accidents[monthly_accidents['distance'] <= distance_threshold]
        # 計算事故數量
        monthly_accident_counts.append(len(nearby_accidents))
        monthly_injury_counts.append(nearby_accidents['injury'].sum())

        # 記錄天氣資訊
        monthly_weather_records.append(nearby_accidents['weather'].tolist())
    
    accident_count.append(monthly_accident_counts)
    injury_count.append(monthly_injury_counts)
    weather_records.append(monthly_weather_records)

# 將結果轉換為 DataFrame
accident_count_df = pd.DataFrame(accident_count, columns=year2months.astype(str))
injury_count_df = pd.DataFrame(injury_count, columns=year2months.astype(str))
weather_records_df = pd.DataFrame(weather_records, columns=year2months.astype(str))

In [7]:
# 號誌數量
sign = pd.read_csv('rawdata/34條主次要幹道標誌1130402.csv')
# 計算全部equipment與sign的個數
sign_distance_results = []
for i, equipment in used_equipment.iterrows():
    # 計算距離並篩選100公尺（0.1公里）內
    sign['distance'] = sign.apply(
        lambda x: haversine((equipment['latitude'], equipment['longitude']), 
                            (x['緯度座標Y-WGS84'], x['經度座標X-WGS84'])),
        axis=1
    )
    nearby_signs = sign[sign['distance'] <= 0.015]
    if nearby_signs.empty:
        # 若無最近標誌，記錄 None
        sign_distance_results.append({'equipment_index': i, 'nearest_sign': None, 'sign_count': 0})
    else:
        # 找到最近的標誌，並計算該路口的標誌數量
        nearest_sign = nearby_signs.loc[nearby_signs['distance'].idxmin()]
        sign_count = nearby_signs[nearby_signs['路口名稱'] == nearest_sign['路口名稱']].shape[0]
        sign_distance_results.append({
            'equipment_index': i,
            'nearest_sign': nearest_sign['路口名稱'],
            'sign_count': sign_count
        })

# 將結果轉換為 DataFrame
sign_distance_results_df = pd.DataFrame(sign_distance_results)

In [8]:
# 天氣資料
weather = pd.read_excel('rawdata/110_112_臺北市氣象站歷史觀測資料.xlsx')
weather.columns = [
    'Year', 'Month', 'year2month', 'TemperatureAverage', 'TemperatureMax', 'TemperatureMin', 
    'Precipitation', 'Wind', 'WindMax', 'HumidityAverage', 'HumidityMin', 
    'Pressure', 'RainDay', 'Sunshine'
]
weather['year2month'] = pd.to_datetime(weather['year2month'], errors='coerce').dt.to_period('M').astype(str)
weather_filtered = weather[['year2month', 'TemperatureAverage', 'RainDay']]

In [9]:
# 車輛資料
carNumber = pd.read_csv('rawdata/臺北市機動車輛登記數按月別_行政區.csv')
carNumber = carNumber[['統計期', '行政區別', '每千人持有汽車數', '每千人持有機車數']]
carNumber.columns = ['year2month', 'district', 'carNumber', 'motorcycleNumber']
year2month = carNumber['year2month'].str.split('年|月', expand=True)
year2month.columns = ['year', 'month', 'other']
year2month['year2monthNew'] = (pd.to_numeric(year2month['year'])+1911).astype(str) + '-' + year2month['month']
year2month['year2monthNew'] = pd.to_datetime(year2month['year2monthNew'], errors='coerce').dt.to_period('M').astype(str)
carNumber = pd.concat([carNumber, year2month], axis=1)
carNumber = carNumber[['year2monthNew', 'district', 'carNumber', 'motorcycleNumber']]
# 計算每個 district 的車輛數
district_results = []
for _, row in used_equipment.iterrows():
    district_list = row['district'].split('、')  # 分割設備的多行政區資料
    year_month_results = []
    for year2month in carNumber['year2monthNew'].unique():
        # 篩選當前月份的車輛數據
        monthly_data = carNumber[carNumber['year2monthNew'] == year2month]
        
        # 計算多個行政區的數據
        total_car_number = 0
        total_motorcycle_number = 0
        for district in district_list:
            matching_data = monthly_data[monthly_data['district'].str.contains(district, na=False)]
            total_car_number += matching_data['carNumber'].sum()
            total_motorcycle_number += matching_data['motorcycleNumber'].sum()
        
        # 保存結果
        year_month_results.append({
            'district': row['district'],
            'carNumberPer': total_car_number,
            'motorcycleNumberPer': total_motorcycle_number,
            'year2Month': year2month
        })
    district_results.extend(year_month_results)
# 轉換為 DataFrame
district_summary_df = pd.DataFrame(district_results)

C:\Users\h2408\AppData\Local\Temp\ipykernel_32668\394329374.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  year2month['year2monthNew'] = pd.to_datetime(year2month['year2monthNew'], errors='coerce').dt.to_period('M').astype(str)


In [10]:
# 合併所有資料
final_results = []
for i, equipment in used_equipment.iterrows():
    for year2month, accident_counts in zip(year2months.astype(str), accident_count_df.iloc[i]):
        record = {
            'address': equipment['address'],
            'district': equipment['district'],
            'longitude': equipment['longitude'],
            'latitude': equipment['latitude'],
            'year': year2month[:4],
            'month': year2month[5:],
            'year2month': year2month,
            'accident_count': accident_counts,
            'injury_count': injury_count_df.iloc[i][year2month],
            # 'sign_count': sign_distance_results_df.loc[sign_distance_results_df['equipment_index'] == i, 'sign_count'].values[0],
            'carNumberPer': district_summary_df.loc[(district_summary_df['district'] == equipment['district']) & (district_summary_df['year2Month'] == year2month), 'carNumberPer'].values[0],
            'motorcycleNumberPer': district_summary_df.loc[(district_summary_df['district'] == equipment['district']) & (district_summary_df['year2Month'] == year2month), 'motorcycleNumberPer'].values[0]
        }
        weather_data = weather_filtered[weather_filtered['year2month'] == year2month]
        if not weather_data.empty:
            record.update(weather_data.iloc[0].to_dict())
        final_results.append(record)

final_results_df = pd.DataFrame(final_results)
final_results_df.head()

,address,district,longitude,latitude,year,month,year2month,accident_count,injury_count,carNumberPer,motorcycleNumberPer,TemperatureAverage,RainDay
0,市民大道與承德路口,中正區、大同區,121.516415,25.048688,2022,06,2022-06,3,5.0,632,825,28.1,15
1,市民大道與承德路口,中正區、大同區,121.516415,25.048688,2022,07,2022-07,8,22.0,632,824,30.8,10
2,市民大道與承德路口,中正區、大同區,121.516415,25.048688,2022,08,2022-08,1,4.0,633,823,30.6,10
3,市民大道與承德路口,中正區、大同區,121.516415,25.048688,2022,09,2022-09,4,6.0,632,824,27.3,15
4,市民大道與承德路口,中正區、大同區,121.516415,25.048688,2022,10,2022-10,2,4.0,631,821,24.2,20


In [11]:
# summary statistics
final_results_df.describe()

,longitude,latitude,accident_count,injury_count,carNumberPer,motorcycleNumberPer,TemperatureAverage,RainDay
count,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,121.529187,25.052814,0.492105,0.805263,466.836842,480.968421,24.657895,11.526316
std,0.022809,0.029502,0.934499,1.848903,239.770206,156.514725,4.760581,3.903431
min,121.471438,24.987827,0.000000,0.000000,220.000000,245.000000,16.600000,5.000000
25%,121.514165,25.034855,0.000000,0.000000,273.500000,382.000000,20.000000,8.000000
50%,121.530302,25.048037,0.000000,0.000000,350.000000,433.500000,25.300000,10.000000
75%,121.549225,25.062994,1.000000,1.000000,647.500000,599.250000,29.000000,15.000000
max,121.564334,25.122991,8.000000,22.000000,965.000000,825.000000,30.800000,20.000000


In [12]:
final_results_df['accident_count'].sum()

187

In [13]:
# save
final_results_df.to_csv(fileName, index=False)